In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataclasses import dataclass

import pandas as pd
import scipy
import scipy.stats
import numpy as np

import price_time_series
from amm import AMM, Option, black_scholes
from users import RandomUser

In [ ]:
%matplotlib inline

# Simulation of happens with liquidity pool

In [ ]:
alpha = 0.3
beta = 0.1

price, volatility = price_time_series.generate_price_variance_process(
    alpha = alpha,
    beta = beta,
    series_len = 1_000
)

In [ ]:
pd.Series(volatility).plot()

# Set up

## Liquidity pool

- for simplicity there is only one maturity and it expires at the end of the time series
- the pool(s) allows for both put and call and also for buying and selling
- there is only one instrument hence one pool for call option and one for put option

## AMM


## Traders

- traders have unlimited capital -> their capital and profit/loss is not measured
- all traders trade the same amount, equal to 1 token
- at each round users are shuffled

### There are 3 arbitragers

- first sees the volatility that generated the process and believes that it should be 10% lower
- second sees the volatility that generated the process and believes it is the true one
- third sees the volatility that generated the process and believes that it should be 10% higher

Each arbitrager executes its trade based on probability given by:

- for short (underwriting) -> (pool_premia - arbitragers_premia) / arbitragers_premia * multiplier
- for long -> (arbitragers_premia - pool_premia) / arbitragers_premia * multiplier

where
- arbitragers_premia is calculated by the volatility that the arbitrager believes is true
- pool_premia is adjusted for fees

### There is a random trader

- Trader that trades option with some probability.

In [ ]:
time_till_maturity_start=900
amm = AMM(time_till_maturity=time_till_maturity_start, current_underlying_price=1.)
random_user = RandomUser(trade_probability=0.6, put_strikes=amm.put_strikes, call_strikes=amm.call_strikes)
users = [
    random_user
]

- For example ETH/USDC the ETH is base and USDC is quote currency.
- Size (quantity) of option is measured in base tokens (ETH).
- Put premia is measured in quote token (USDC), call premia in base (ETH) token.
- The quote (USDC) pool is used for put options and the base (ETH) pool for call options.

In [ ]:
# price, volatility
for current_price, time_till_maturity in zip(price[100:100+time_till_maturity_start], range(time_till_maturity_start, 0, -1)):
    amm.next_epoch(time_till_maturity=time_till_maturity, current_underlying_price=current_price)
    
    # in each epoch the users are randomly ordered
    np.random.shuffle(users)
    for user in users:
        trade = user.trade(current_price)
        if trade is not None:
            amm.trade(
                strike_price=trade['strike_price'],
                type_=trade['type_'],
                long_short=trade['long_short'],
                quantity=trade['quantity']
            )

In [ ]:
amm.clear()

In [ ]:
amm.__dict__()

In [ ]:
#     def next_epoch(self, time_till_maturity: int, current_underlying_price: float) -> None:
#     def get_premia(self, strike_price: float, type_: str, long_short: str, quantity: float = 1.) -> float:
#     def trade(self, strike_price: float, type_: str, long_short: str, quantity: float) -> Option:

# Specific examples (step by step)

In [ ]:
(
    amm.get_premia(strike_price=1.1, type_='call', long_short='long', quantity=1.),
    amm.get_premia(strike_price=1.1, type_='call', long_short='short', quantity=1.),
    amm.get_premia(strike_price=.9, type_='put', long_short='long', quantity=1.),
    amm.get_premia(strike_price=.9, type_='put', long_short='short', quantity=1.)
)

In [ ]:
amm.next_epoch(time_till_maturity=10, current_underlying_price=1)

In [ ]:
(
    amm.get_premia(strike_price=1.2, type_='call', long_short='long', quantity=1.),
    amm.get_premia(strike_price=1.2, type_='call', long_short='short', quantity=1.),
    amm.get_premia(strike_price=.8, type_='put', long_short='long', quantity=1.),
    amm.get_premia(strike_price=.8, type_='put', long_short='short', quantity=1.)
)

In [ ]:
user_option_1 = amm.trade(strike_price=1.2, type_='call', long_short='long', quantity=1.)
print(user_option_1)
print(
    f"""
        call_volatility: {amm.call_volatility}
        put_volatility: {amm.put_volatility}
        call_pool_size: {amm.call_pool_size}
        put_pool_size: {amm.put_pool_size}
        call_issued_options: {amm.call_issued_options}
        put_issued_options: {amm.put_issued_options}
        time_till_maturity: {amm.time_till_maturity}
        current_underlying_price: {amm.current_underlying_price}
    """
)

In [ ]:
user_option_1 = amm.trade(strike_price=0.9, type_='put', long_short='short', quantity=1.)
print(user_option_1)
print(
    f"""
        call_volatility: {amm.call_volatility}
        put_volatility: {amm.put_volatility}
        call_pool_size: {amm.call_pool_size}
        put_pool_size: {amm.put_pool_size}
        call_issued_options: {amm.call_issued_options}
        put_issued_options: {amm.put_issued_options}
        time_till_maturity: {amm.time_till_maturity}
        current_underlying_price: {amm.current_underlying_price}
    """
)

In [ ]:
user_option_1 = amm.trade(strike_price=0.9, type_='put', long_short='long', quantity=1.)
print(user_option_1)
print(
    f"""
        call_volatility: {amm.call_volatility}
        put_volatility: {amm.put_volatility}
        call_pool_size: {amm.call_pool_size}
        put_pool_size: {amm.put_pool_size}
        call_issued_options: {amm.call_issued_options}
        put_issued_options: {amm.put_issued_options}
        time_till_maturity: {amm.time_till_maturity}
        current_underlying_price: {amm.current_underlying_price}
    """
)

In [ ]:
user_option_1 = amm.trade(strike_price=1.2, type_='call', long_short='short', quantity=1.)
print(user_option_1)
print(
    f"""
        call_volatility: {amm.call_volatility}
        put_volatility: {amm.put_volatility}
        call_pool_size: {amm.call_pool_size}
        put_pool_size: {amm.put_pool_size}
        call_issued_options: {amm.call_issued_options}
        put_issued_options: {amm.put_issued_options}
        time_till_maturity: {amm.time_till_maturity}
        current_underlying_price: {amm.current_underlying_price}
    """
)